In [1]:
import pandas as pd
import numpy as np
import spacy
import joblib
import os
from imdb import IMDb
from tqdm import tqdm
import requests
from scipy import sparse

In [2]:
data_total = pd.read_csv('../data/dataset_totalL.csv')
data_total.head()

C:\Users\Javier\AppData\Local\Temp\ipykernel_1068\3151481919.py:1: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  data_total = pd.read_csv('../data/dataset_totalL.csv')


,title,vote_average,vote_count,release_date,adult,original_language,overview,popularity,tagline,genres,...,production_countries,spoken_languages,keywords,director,top_actors,runtime_final,budget_final,overview_clean,title_clean,tagline_clean
0,inception,8.364,34495,2010-07-15,False,en,"cobb, a skilled thief who commits corporate es...",83.952,your mind is the scene of the crime.,"Action, Science Fiction, Adventure",...,"United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc...",christopher nolan,"leonardo dicaprio, joseph gordon-levitt, ken w...",148.0,160000000.0,cobb thief commit espionage infiltrate subcons...,inception,mind scene crime
1,interstellar,8.417,32571,2014-11-05,False,en,the adventures of a group of explorers who mak...,140.241,mankind was born on earth. it was never meant ...,"Adventure, Drama, Science Fiction",...,"United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,...",christopher nolan,"matthew mcconaughey, anne hathaway, michael ca...",169.0,165000000.0,adventure group explorer use discover wormhole...,interstellar,mankind bear earth mean die
2,the dark knight,8.512,30619,2008-07-16,False,en,batman raises the stakes in his war on crime. ...,130.643,welcome to a world without rules.,"Drama, Action, Crime, Thriller",...,"United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f...",christopher nolan,"christian bale, heath ledger, aaron eckhart, m...",152.0,185000000.0,batman raise stake war crime help lt jim gordo...,dark knight,welcome world rule
3,avatar,7.573,29815,2009-12-15,False,en,"in the 22nd century, a paraplegic marine is di...",79.932,enter the world of pandora.,"Action, Adventure, Fantasy, Science Fiction",...,"United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ...",james cameron,"sam worthington, zoe saldaña, sigourney weaver...",162.0,237000000.0,century marine dispatch moon pandora mission t...,avatar,enter world pandora
4,the avengers,7.710,29166,2012-04-25,False,en,when an unexpected enemy emerges and threatens...,98.082,some assembly required.,"Science Fiction, Action, Adventure",...,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com...",joss whedon,"robert downey jr., chris evans, mark ruffalo, ...",143.0,220000000.0,enemy emerge threaten safety security nick fur...,avenger,assembly require


In [3]:
# Eliminar filas que tengan el género 'Documentary' en df_posters
data_total = data_total[~data_total['genres'].str.contains('Documentary', case=False, na=False)]
data_total = data_total[data_total['runtime_final'] > 30]
palabras_clave = ['porn', 'porno', 'adult', 'xxx', 'erotic', 'sex','parody','wwe ','wrestling','wrestler','wrestlers','wrestlemania','wwe raw','wwe smackdown','wwe nxt','wwf','wwesuperstar','wwesuperstars','wcw']
patron = '|'.join(palabras_clave)
# Contar cuántos registros contienen las palabras clave en al menos una columna
mask = (
    data_total['title'].str.contains(patron, case=False, na=False) |
    data_total['genres'].str.contains(patron, case=False, na=False) |
    data_total['overview'].str.contains(patron, case=False, na=False)
)
cantidad = mask.sum()
print(f"Registros con palabras clave (porno/adulto): {cantidad}")
data_total = data_total[~mask]

Registros con palabras clave (porno/adulto): 6490


In [4]:
data_total.shape

(93494, 21)

In [5]:
import pandas as pd
import spacy
from tqdm import tqdm
import ast

# Cargar spaCy sin parser ni NER
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
tqdm.pandas()

# Rellenar nulos y convertir a minúsculas
cols_to_lower = [
    'keywords', 'tagline', 'title', 'overview', 'top_actors', 'director',
    'production_companies', 'genres', 'original_language',
    'spoken_languages', 'production_countries'
]
for col in cols_to_lower:
    data_total[col] = data_total[col].fillna('').str.lower()

# Función para limpieza semántica con spaCy
def clean_text_spacy(text, pos_tags):
    doc = nlp(text)
    return " ".join(
        token.lemma_ for token in doc
        if token.pos_ in pos_tags and token.is_alpha and not token.is_stop
    )

# Aplicar limpieza a overview, title y tagline
data_total['overview_clean'] = data_total['overview'].progress_apply(lambda x: clean_text_spacy(x, ['NOUN', 'VERB', 'PROPN', 'ADJ']))
data_total['title_clean'] = data_total['title'].progress_apply(lambda x: clean_text_spacy(x, ['NOUN', 'VERB', 'PROPN', 'ADJ']))
data_total['tagline_clean'] = data_total['tagline'].progress_apply(lambda x: clean_text_spacy(x, ['NOUN', 'VERB', 'PROPN', 'ADJ']))

# Función para limpiar columnas tipo lista como texto
def process_list_column(text):
    try:
        items = ast.literal_eval(text) if isinstance(text, str) and text.startswith('[') else text.split(',')
    except:
        items = text.split(',')
    return " ".join(sorted(set(i.strip().strip("'\" ") for i in items if i.strip())))

list_like_cols = ['keywords', 'top_actors', 'director', 'production_companies',
                  'genres', 'original_language', 'spoken_languages', 'production_countries']
for col in list_like_cols:
    data_total[col + '_clean'] = data_total[col].progress_apply(process_list_column)

# Eliminar duplicados de palabras en cada fila
def remove_duplicates(text):
    return " ".join(dict.fromkeys(text.split()))

for col in ['title_clean', 'overview_clean', 'tagline_clean'] + [c + '_clean' for c in list_like_cols]:
    data_total[col] = data_total[col].progress_apply(remove_duplicates)


100%|██████████| 93494/93494 [00:00<00:00, 738134.85it/s]


In [6]:
data_total.head()

,title,vote_average,vote_count,release_date,adult,original_language,overview,popularity,tagline,genres,...,title_clean,tagline_clean,keywords_clean,top_actors_clean,director_clean,production_companies_clean,genres_clean,original_language_clean,spoken_languages_clean,production_countries_clean
0,inception,8.364,34495,2010-07-15,False,en,"cobb, a skilled thief who commits corporate es...",83.952,your mind is the scene of the crime.,"action, science fiction, adventure",...,inception,mind scene crime,airplane allegory architecture california car ...,berenger cillian murphy dileep rao elliot page...,christopher nolan,legendary pictures syncopy warner bros.,action adventure science fiction,en,english french japanese swahili,united kingdom states of america
1,interstellar,8.417,32571,2014-11-05,False,en,the adventures of a group of explorers who mak...,140.241,mankind was born on earth. it was never meant ...,"adventure, drama, science fiction",...,interstellar,mankind bear earth mean die,2060s artificial intelligence (a.i.) astronaut...,anne hathaway casey affleck ellen burstyn jess...,christopher nolan,legendary pictures lynda obst productions syncopy,adventure drama science fiction,en,english,united kingdom states of america
2,the dark knight,8.512,30619,2008-07-16,False,en,batman raises the stakes in his war on crime. ...,130.643,welcome to a world without rules.,"drama, action, crime, thriller",...,dark knight,welcome world rule,anti hero villain based on comic chaos crime f...,aaron eckhart christian bale cillian murphy ga...,christopher nolan,dc comics isobel griffiths legendary pictures ...,action crime drama thriller,en,english mandarin,united kingdom states of america
3,avatar,7.573,29815,2009-12-15,False,en,"in the 22nd century, a paraplegic marine is di...",79.932,enter the world of pandora.,"action, adventure, fantasy, science fiction",...,avatar,enter world pandora,alien planet anti war battle culture clash fut...,cch pounder giovanni ribisi joel david moore l...,james cameron,dune entertainment ingenious media lightstorm ...,action adventure fantasy science fiction,en,english spanish,united kingdom states of america
4,the avengers,7.710,29166,2012-04-25,False,en,when an unexpected enemy emerges and threatens...,98.082,some assembly required.,"science fiction, action, adventure",...,avenger,assembly require,aftercreditsstinger alien invasion based on co...,chris evans clark gregg cobie smulders hemswor...,joss whedon,marvel studios,action adventure science fiction,en,english hindi russian,united states of america


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
from scipy import sparse

columnas_vectorizadas = ['keywords_clean', 'director_clean', 'top_actors_clean', 'production_companies_clean', 'overview_clean', 'title_clean', 'tagline_clean', 'original_language_clean', 'spoken_languages_clean', 'genres_clean', 'production_countries_clean']

for col in columnas_vectorizadas:
    print(f"Vectorizando columna: {col}")

    # 1. Preparar textos completos y no vacíos
    texts_all = data_total[col].fillna('').astype(str)
    texts_nonempty = texts_all[texts_all.str.strip() != '']

    if texts_nonempty.empty:
        print(f"  ❌ Todos los textos en {col} están vacíos. Se omite.")
        continue

    # 2. Fit solo con no vacíos
    vectorizer = TfidfVectorizer(max_features=10000, lowercase=True)
    vectorizer.fit(texts_nonempty)

    # 3. Transformar todos (se conserva el índice)
    vect = vectorizer.transform(texts_all)
   
    sparse.save_npz(f"../data/{col}.npz", vect)
    joblib.dump(vectorizer, f"../models/{col}_vect.pkl")

Vectorizando columna: keywords_clean
Vectorizando columna: director_clean
Vectorizando columna: top_actors_clean
Vectorizando columna: production_companies_clean
Vectorizando columna: overview_clean
Vectorizando columna: title_clean
Vectorizando columna: tagline_clean
Vectorizando columna: original_language_clean
Vectorizando columna: spoken_languages_clean
Vectorizando columna: genres_clean
Vectorizando columna: production_countries_clean


In [8]:
data_total['adult'] = data_total['adult'].astype(int)  # True → 1, False → 0

In [9]:
data_total['release_date'] = pd.to_datetime(data_total['release_date'], errors='coerce')
data_total['release_year'] = data_total['release_date'].dt.year.fillna(0).astype(int)
data_total.drop(columns=['release_date'], inplace=True) 

In [10]:
data_total.head()

,title,vote_average,vote_count,adult,original_language,overview,popularity,tagline,genres,production_companies,...,tagline_clean,keywords_clean,top_actors_clean,director_clean,production_companies_clean,genres_clean,original_language_clean,spoken_languages_clean,production_countries_clean,release_year
0,inception,8.364,34495,0,en,"cobb, a skilled thief who commits corporate es...",83.952,your mind is the scene of the crime.,"action, science fiction, adventure","legendary pictures, syncopy, warner bros. pict...",...,mind scene crime,airplane allegory architecture california car ...,berenger cillian murphy dileep rao elliot page...,christopher nolan,legendary pictures syncopy warner bros.,action adventure science fiction,en,english french japanese swahili,united kingdom states of america,2010
1,interstellar,8.417,32571,0,en,the adventures of a group of explorers who mak...,140.241,mankind was born on earth. it was never meant ...,"adventure, drama, science fiction","legendary pictures, syncopy, lynda obst produc...",...,mankind bear earth mean die,2060s artificial intelligence (a.i.) astronaut...,anne hathaway casey affleck ellen burstyn jess...,christopher nolan,legendary pictures lynda obst productions syncopy,adventure drama science fiction,en,english,united kingdom states of america,2014
2,the dark knight,8.512,30619,0,en,batman raises the stakes in his war on crime. ...,130.643,welcome to a world without rules.,"drama, action, crime, thriller","dc comics, legendary pictures, syncopy, isobel...",...,welcome world rule,anti hero villain based on comic chaos crime f...,aaron eckhart christian bale cillian murphy ga...,christopher nolan,dc comics isobel griffiths legendary pictures ...,action crime drama thriller,en,english mandarin,united kingdom states of america,2008
3,avatar,7.573,29815,0,en,"in the 22nd century, a paraplegic marine is di...",79.932,enter the world of pandora.,"action, adventure, fantasy, science fiction","dune entertainment, lightstorm 20th century fo...",...,enter world pandora,alien planet anti war battle culture clash fut...,cch pounder giovanni ribisi joel david moore l...,james cameron,dune entertainment ingenious media lightstorm ...,action adventure fantasy science fiction,en,english spanish,united kingdom states of america,2009
4,the avengers,7.710,29166,0,en,when an unexpected enemy emerges and threatens...,98.082,some assembly required.,"science fiction, action, adventure",marvel studios,...,assembly require,aftercreditsstinger alien invasion based on co...,chris evans clark gregg cobie smulders hemswor...,joss whedon,marvel studios,action adventure science fiction,en,english hindi russian,united states of america,2012


In [11]:
data_total.columns

Index(['title', 'vote_average', 'vote_count', 'adult', 'original_language',
       'overview', 'popularity', 'tagline', 'genres', 'production_companies',
       'production_countries', 'spoken_languages', 'keywords', 'director',
       'top_actors', 'runtime_final', 'budget_final', 'overview_clean',
       'title_clean', 'tagline_clean', 'keywords_clean', 'top_actors_clean',
       'director_clean', 'production_companies_clean', 'genres_clean',
       'original_language_clean', 'spoken_languages_clean',
       'production_countries_clean', 'release_year'],
      dtype='object')

In [12]:
def limpiar_budget(budget):
    if pd.isna(budget):
        return np.nan
    # Eliminar todo lo que no sea dígito
    limpio = ''.join(c for c in str(budget) if c.isdigit())
    return float(limpio) if limpio else np.nan

# Aplica la limpieza
data_total['budget_final'] = data_total['budget_final'].apply(limpiar_budget) / 1_000_000  # En millones

In [13]:
data_total.drop(['title', 'overview', 'tagline', 'keywords'], axis=1, inplace=True)

In [14]:
tit_post = pd.read_csv('../data/data3_streamlit_titles.csv')
# Eliminar filas que tengan el género 'Documentary' en df_posters
tit_post = tit_post[~tit_post['genres'].str.contains('Documentary', case=False, na=False)]
tit_post = tit_post[tit_post['runtime_final'] > 30]
palabras_clave = ['porn', 'porno', 'adult', 'xxx', 'erotic', 'sex','parody','wwe ','wrestling','wrestler','wrestlers','wrestlemania','wwe raw','wwe smackdown','wwe nxt','wwf','wwesuperstar','wwesuperstars','wcw']
patron = '|'.join(palabras_clave)
# Contar cuántos registros contienen las palabras clave en al menos una columna
mask = (
    tit_post['title'].str.contains(patron, case=False, na=False) |
    tit_post['genres'].str.contains(patron, case=False, na=False) |
    tit_post['overview'].str.contains(patron, case=False, na=False)
)
cantidad = mask.sum()
print(f"Registros con palabras clave (porno/adulto): {cantidad}")
tit_post = tit_post[~mask]
tit_post = tit_post[['title', 'poster_path']]
tit_post.head()

C:\Users\Javier\AppData\Local\Temp\ipykernel_1068\2120689358.py:1: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  tit_post = pd.read_csv('../data/data3_streamlit_titles.csv')


Registros con palabras clave (porno/adulto): 6490


,title,poster_path
0,inception,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg
1,interstellar,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg
2,the dark knight,/qJ2tW6WMUDux911r6m7haRef0WH.jpg
3,avatar,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg
4,the avengers,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg


In [15]:
data_total = pd.concat([tit_post, data_total], axis=1)
data_total.head()

,title,poster_path,vote_average,vote_count,adult,original_language,popularity,genres,production_companies,production_countries,...,tagline_clean,keywords_clean,top_actors_clean,director_clean,production_companies_clean,genres_clean,original_language_clean,spoken_languages_clean,production_countries_clean,release_year
0,inception,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,8.364,34495,0,en,83.952,"action, science fiction, adventure","legendary pictures, syncopy, warner bros. pict...","united kingdom, united states of america",...,mind scene crime,airplane allegory architecture california car ...,berenger cillian murphy dileep rao elliot page...,christopher nolan,legendary pictures syncopy warner bros.,action adventure science fiction,en,english french japanese swahili,united kingdom states of america,2010
1,interstellar,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,8.417,32571,0,en,140.241,"adventure, drama, science fiction","legendary pictures, syncopy, lynda obst produc...","united kingdom, united states of america",...,mankind bear earth mean die,2060s artificial intelligence (a.i.) astronaut...,anne hathaway casey affleck ellen burstyn jess...,christopher nolan,legendary pictures lynda obst productions syncopy,adventure drama science fiction,en,english,united kingdom states of america,2014
2,the dark knight,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,8.512,30619,0,en,130.643,"drama, action, crime, thriller","dc comics, legendary pictures, syncopy, isobel...","united kingdom, united states of america",...,welcome world rule,anti hero villain based on comic chaos crime f...,aaron eckhart christian bale cillian murphy ga...,christopher nolan,dc comics isobel griffiths legendary pictures ...,action crime drama thriller,en,english mandarin,united kingdom states of america,2008
3,avatar,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,7.573,29815,0,en,79.932,"action, adventure, fantasy, science fiction","dune entertainment, lightstorm 20th century fo...","united states of america, united kingdom",...,enter world pandora,alien planet anti war battle culture clash fut...,cch pounder giovanni ribisi joel david moore l...,james cameron,dune entertainment ingenious media lightstorm ...,action adventure fantasy science fiction,en,english spanish,united kingdom states of america,2009
4,the avengers,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,7.710,29166,0,en,98.082,"science fiction, action, adventure",marvel studios,united states of america,...,assembly require,aftercreditsstinger alien invasion based on co...,chris evans clark gregg cobie smulders hemswor...,joss whedon,marvel studios,action adventure science fiction,en,english hindi russian,united states of america,2012


In [16]:
data_total.to_csv('../data/data3_streamlit_titles.csv', index=False)

In [17]:
from sklearn.preprocessing import MinMaxScaler

# Columnas numéricas
numeric_cols = ['vote_average', 'vote_count', 'adult', 'popularity', 'runtime_final', 'budget_final', 'release_year']

# Función de limpieza para valores monetarios o con texto
def clean_numeric(value):
    if pd.isna(value):
        return 0
    if isinstance(value, str):
        # Eliminar cualquier carácter no numérico o punto decimal
        value = ''.join(c for c in value if c.isdigit() or c == '.')
        try:
            return float(value)
        except:
            return 0
    return value  # Si ya es numérico

# Aplicar limpieza a todas las columnas
for col in numeric_cols:
    data_total[col] = data_total[col].apply(clean_numeric)

# Rellenar NaNs (por si acaso)
data_total[numeric_cols] = data_total[numeric_cols].fillna(0)

# Escalar
scaler = MinMaxScaler()
data_total[numeric_cols] = scaler.fit_transform(data_total[numeric_cols])

# Guardar el escalador
joblib.dump(scaler, "../models/numeric_minmax_scaler.pkl")

['../models/numeric_minmax_scaler.pkl']

In [18]:
data_total.to_csv('../data/dataset_escalado.csv', index=False)

In [20]:
from scipy.sparse import hstack, csr_matrix

# Elimina columnas tipo object (como textos, listas, etc.)
df_numeric = data_total.select_dtypes(include=['number', 'Sparse'])

# Convierte solo las columnas numéricas a sparse
X_num_sparse = csr_matrix(df_numeric.values)

In [21]:
import pandas as pd
from scipy.sparse import csr_matrix
# Paso 1: Selecciona las columnas numéricas que quieres usar
numeric_cols = ['vote_average', 'vote_count', 'adult', 'popularity', 'runtime_final', 'budget_final', 'release_year']
# Paso 2: Extrae esas columnas del DataFrame principal
data_total = data_total[numeric_cols].copy()
# Paso 3: Fuerza a que todo sea numérico (por si hay strings u objetos)
data_total = data_total.apply(pd.to_numeric, errors='coerce')
# Paso 4: Llena valores nulos (por errores o valores faltantes) con 0
data_total = data_total.fillna(0)
# Paso 5: Convierte a float32 para ahorrar RAM y pásalo a matriz dispersa
X_num_sparse = csr_matrix(data_total.astype('float32'))
# Ahora X_num_sparse está listo para combinar con otros vectores

In [22]:
print(X_num_sparse.shape)

(93494, 7)


In [23]:
tit_post.shape

(93494, 2)

In [24]:
data_total.shape

(93494, 7)

In [25]:
from scipy.sparse import load_npz, save_npz, hstack
from scipy.sparse import csr_matrix

# Cargar las matrices sparse desde disco
title = load_npz('../data/title_clean.npz')
tagline = load_npz('../data/tagline_clean.npz')
actors = load_npz('../data/top_actors_clean.npz')
companies = load_npz('../data/production_companies_clean.npz')
overview = load_npz('../data/overview_clean.npz')
keywords = load_npz('../data/keywords_clean.npz')
director = load_npz('../data/director_clean.npz')
genres = load_npz('../data/genres_clean.npz')
orig_lang = load_npz('../data/original_language_clean.npz')
countries = load_npz('../data/production_countries_clean.npz')
spoken_lang = load_npz('../data/spoken_languages_clean.npz')

# Convertir tus datos numéricos a sparse
X_num_sparse = csr_matrix(data_total)

# Combinar todo
X_combined = hstack([
    title, tagline, actors, companies,
    overview, keywords, director, genres, orig_lang, countries, spoken_lang,
    X_num_sparse
])


In [26]:
print(X_combined.shape)

(93494, 70581)


In [27]:
save_npz('../data/matrix_combinada.npz', X_combined)

In [28]:
from sklearn.neighbors import NearestNeighbors
import time  # solo para simular retraso, quitar en producción

# Supongamos que preparar X_combined es un proceso con loop
# Aquí simulo con un loop dummy para mostrar tqdm
for _ in tqdm(range(100), desc='Entrenando NearestNeighbors'):
    time.sleep(0.01)  # simula trabajo pesado

# Aquí ya tienes X_combined preparado, entrenas el modelo normalmente
model = NearestNeighbors(n_neighbors=10, metric='cosine')
model.fit(X_combined)

joblib.dump(model, '../models/knn_model.pkl')

Entrenando NearestNeighbors: 100%|██████████| 100/100 [00:01<00:00, 64.42it/s]


['../models/knn_model.pkl']

In [29]:
print("Modelo espera:", model._fit_X.shape[1])

Modelo espera: 70581


In [30]:
import random

df = pd.read_csv('../data/dataset_escalado.csv')
model = joblib.load('../models/knn_model.pkl')
X_combined = sparse.load_npz('../data/matrix_combinada.npz')

def recomendar_pelicula(titulo, modelo_knn, X, df, n=10):
    try:
        idx = df[df['title_clean'] == titulo].index[0]
    except IndexError:
        return f"❌ Título '{titulo}' no encontrado."

    distancias, indices = modelo_knn.kneighbors(X[idx], n_neighbors=n+1)
    recomendaciones = df.iloc[indices[0][1:]]['title_clean'].values
    return recomendaciones

# ---------------------------
# 8. Probar con títulos aleatorios
# ---------------------------
titulos_random = df['title_clean'].unique()
titulo = 'prestige'

print("🎬 Título seleccionado:", titulo)
recs = recomendar_pelicula(titulo, model, X_combined, df)

print("\n🔁 Recomendaciones:")
for r in recs:
    print("  ✅", r)

🎬 Título seleccionado: prestige

🔁 Recomendaciones:
  ✅ interstellar
  ✅ tenet
  ✅ follow
  ✅ inception
  ✅ dark knight rise
  ✅ afterschool
  ✅ memento
  ✅ nan
  ✅ nan
  ✅ evolution
